In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity 
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional,Dense,GlobalMaxPooling1D,LSTM,Embedding
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [2]:
data = pd.read_csv("../../Data/clean_data.csv")

In [3]:
bert_preproccessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# Using a smaller model replace it with a bigger model
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")

In [4]:
def create_embedding(sentence):
    preroccessed_sentence = bert_preproccessor(sentence)
    bert_embedding = bert_model(preroccessed_sentence)
    return bert_embedding["sequence_output"]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(data["Tweet Text"],data["Informativeness"])

In [9]:
# Defining the model

input = keras.Input(shape=(),dtype="string",name="input")

layer = Bidirectional(CuDNNLSTM(128,return_sequences=True))(create_embedding(input))
layer = Bidirectional(CuDNNLSTM(128,return_sequences=True))(layer)
layer = GlobalMaxPooling1D()(layer)
layer = Dense(512, activation="relu")(layer)
layer = Dense(1, activation="sigmoid")(layer)
model = keras.Model(inputs=[input],outputs=layer)


model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(X_train,y_train,epochs=5)
model.summary()

Epoch 1/5
24/24 [==============================] - 6s 102ms/step - loss: 0.4045 - accuracy: 0.8187
Epoch 2/5
24/24 [==============================] - 2s 100ms/step - loss: 0.2775 - accuracy: 0.8867
Epoch 3/5
24/24 [==============================] - 2s 102ms/step - loss: 0.2177 - accuracy: 0.8987
Epoch 4/5
24/24 [==============================] - 2s 101ms/step - loss: 0.1853 - accuracy: 0.9280
Epoch 5/5
24/24 [==============================] - 2s 101ms/step - loss: 0.1843 - accuracy: 0.9227
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           input[0][0]                

In [11]:
model.evaluate(X_test,y_test,verbose=2)

8/8 - 2s - loss: 0.2735 - accuracy: 0.9120


[0.2734609544277191, 0.9120000004768372]